In [1]:
import os
import sys
import gensim
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import pickle
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout

ROOT = os.path.dirname(os.getcwd())
#path_data = os.path.join(ROOT, 'data')
path_data = 'C:\\Users\\joris\\Documents\\eScience_data\\data'
sys.path.insert(0, ROOT)
sys.path.insert(0, "C:\\Users\\joris\\Documents\\eScience_data\\spec2vec_gnps_data_analysis\\custom_functions")

In [2]:
sys.path.insert(0, "C:\\Users\\joris\\Documents\\eScience_data\\ms2query\\ms2query")
import s2v_functions
import utils
import networking

In [ ]:
outfile = os.path.join(path_data, 'testing_query_library_s2v_2dec.pickle')
print(outfile)
if os.path.exists(outfile):
    with open(outfile, 'rb') as inf:
        testing_query_library_s2v_2dec = pickle.load(inf)
        new_and_unique2_documents_query_s2v_2dec, new_and_unique2_documents_library_s2v_2dec = testing_query_library_s2v_2dec
else:
    with open(outfile, 'wb') as outf:
        testing_query_library_s2v_2dec = (new_and_unique2_documents_query_s2v_2dec, new_and_unique2_documents_library_s2v_2dec)
        pickle.dump(testing_query_library_s2v_2dec, outf)

In [ ]:
from matchms.importing import load_from_json
outfile = os.path.join(path_data, 'gnps_positive_ionmode_cleaned_by_matchms_and_lookups.pickle')
print(outfile)
if os.path.exists(outfile):
    with open(outfile, 'rb') as inf:
        spectrums = pickle.load(inf)
else:
    filename = os.path.join(path_data,'gnps_positive_ionmode_cleaned_by_matchms_and_lookups.json')
    spectrums = load_from_json(filename)
    with open(outfile, 'wb') as outf:
        pickle.dump(spectrums, outf)

print("number of spectra:", len(spectrums))

In [ ]:
#take this one as example for dummy_app. it's staurosporine!
print(new_and_unique2_documents_query_s2v_2dec[2]._obj.metadata.get('spectrum_id'))
print([new_and_unique2_documents_library_s2v_2dec[x]._obj.metadata.get('spectrum_id') for x in [22101, 20538, 22454, 1366, 65357]])
nn_tested_found_matches_s2v_3_scores_top40[2].iloc[:10]

In [ ]:
testspec_query_id = new_and_unique2_documents_query_s2v_2dec[2]._obj.metadata.get('spectrum_id')
testspec_lib_ids = set([new_and_unique2_documents_library_s2v_2dec[x]._obj.metadata.get('spectrum_id') \
                        for x in [22101, 20538, 22454, 1366, 65357]])
#get unfiltered spectra data
testspec_libs = []
for spec in spectrums:
    if spec.metadata.get('spectrum_id') == testspec_query_id:
        testspec_query = spec
    elif spec.metadata.get('spectrum_id') in testspec_lib_ids:
        testspec_libs.append(spec)

In [ ]:
from matchms.Spectrum import Spectrum
isinstance(testspec_query, Spectrum)

In [ ]:
from matchms.exporting import save_as_json
testspec_query_out = os.path.join(path_data, 'testspectrum_query.json')
save_as_json(testspec_query, testspec_query_out)

testspec_libs_out = os.path.join(path_data, 'testspectrum_library.json')
save_as_json(testspec_libs, testspec_libs_out)

In [ ]:
# example_found_matches = new_and_unique2_found_matches_s2v_2dec[2].loc[[22101, 20538, 22454, 1366, 65357], :]
example_found_matches_file = "C:\\Users\\joris\\Documents\\eScience_data\\data\\test_found_matches.csv"
example_found_matches.to_csv(example_found_matches_file)
example_found_matches

In [4]:
example_found_matches_file = "C:\\Users\\joris\\Documents\\eScience_data\\data\\test_found_matches.csv"
example_found_matches = pd.read_csv(example_found_matches_file, index_col=0)
example_found_matches

,cosine_score,cosine_matches,mod_cosine_score,mod_cosine_matches,s2v_score
22101,0.928457,15,0.928457,15,0.909346
20538,0.856661,11,0.856661,11,0.859138
22454,0.857507,11,0.857507,11,0.674490
1366,0.389183,11,0.389183,11,0.584506
65357,0.090625,3,0.108395,5,0.277905


In [3]:
testspec_query_out = os.path.join(path_data, 'testspectrum_query.json')
testspec_libs_out = os.path.join(path_data, 'testspectrum_library.json')
test_query = utils.json_loader(open(testspec_query_out))
test_library = utils.json_loader(open(testspec_libs_out))
test_doc_query = s2v_functions.process_spectrums(test_query)
test_doc_library = s2v_functions.process_spectrums(test_library)

In [4]:
[doc._obj.get("spectrumid") for doc in test_doc_library]

['CCMSLIB00000223876',
 'CCMSLIB00003136327',
 'CCMSLIB00003138082',
 'CCMSLIB00003138470',
 'CCMSLIB00000566496']

In [9]:
from spec2vec.model_building import train_new_word2vec_model

path_models = os.path.join(path_data, "trained_models")
model_file = os.path.join(path_models, "testspectrum_library_model.model")

iterations = [1, 3, 5, 10, 15, 20]

# Train model with default parameters
model = train_new_word2vec_model(test_doc_library, iterations, model_file)

  Epoch 1 of 20.Change in loss after epoch 1: 4553.359375
Saving model with name: C:\Users\joris\Documents\eScience_data\data\trained_models\testspectrum_library_model_iter_1.model
  Epoch 2 of 20.Change in loss after epoch 2: 4556.564453125
  Epoch 3 of 20.Change in loss after epoch 3: 4534.333984375
Saving model with name: C:\Users\joris\Documents\eScience_data\data\trained_models\testspectrum_library_model_iter_3.model
  Epoch 4 of 20.Change in loss after epoch 4: 4520.9140625
  Epoch 5 of 20.Change in loss after epoch 5: 4530.05078125
Saving model with name: C:\Users\joris\Documents\eScience_data\data\trained_models\testspectrum_library_model_iter_5.model
  Epoch 6 of 20.Change in loss after epoch 6: 4551.943359375
  Epoch 7 of 20.Change in loss after epoch 7: 4555.474609375
  Epoch 8 of 20.Change in loss after epoch 8: 4534.7578125
  Epoch 9 of 20.Change in loss after epoch 9: 4544.56640625
  Epoch 10 of 20.Change in loss after epoch 10: 4544.50390625
Saving model with name: C:\Us

In [6]:
path_models = os.path.join(path_data, "trained_models")
model_file = os.path.join(path_models, "testspectrum_library_model.model")
model = gensim.models.Word2Vec.load(model_file)

In [7]:
lib_length = len(test_doc_library)
test_found_matches = s2v_functions.library_matching(test_doc_query, test_doc_library, model,
                                                    presearch_based_on=[f"spec2vec-top{lib_length}"],
                                                    **{"allowed_missing_percentage": 100})

Pre-selection includes spec2vec top 5.
Found 15 word(s) missing in the model. Weighted missing percentage not covered by the given model is 20.28%.


In [9]:
test_found_matches[0]

,cosine_score,cosine_matches,mod_cosine_score,mod_cosine_matches,s2v_score
0,0.454149,11,0.454149,11,0.999996
1,0.968071,11,0.968071,11,0.999999
2,0.991902,15,0.991902,15,1.000000
3,0.889310,11,0.889310,11,0.999996
4,0.117474,3,0.140509,5,0.999997
